In [1]:
import torch
from torch import nn

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model=torch.load("00000189-checkpoint.pth.tar")
pretrained = {k: v for k, v in model.items()}
print(pretrained.keys())

dict_keys(['generator', 'discriminator', 'kp_detector', 'he_estimator', 'optimizer_generator', 'optimizer_discriminator', 'optimizer_kp_detector', 'optimizer_he_estimator', 'epoch'])


In [4]:
x=model["generator"]

In [6]:
#print(model)

Weight Quantization Calculation.

In [6]:
def weight_quantization(b):

    def uniform_quant(x, b):
        xdiv = x.mul((2**b - 1))
        xhard = xdiv.round().div(2**b - 1)
        return xhard

    class uq(torch.autograd.Function):

        def forward(ctx, input, alpha):
            input_d = input/alpha                          # weights are first divided by alpha
            input_c = input_d.clamp(min=-1, max=1)       # then clipped to [-1,1]
            sign = input_c.sign()
            input_abs = input_c.abs()
            input_q = uniform_quant(input_abs, b).mul(sign)
            ctx.save_for_backward(input, input_q)
            input_q_out = input_q.mul(alpha)               # rescale to the original range
            return input_q_out

    return uq().apply

Quantization function.

In [7]:
class weight_quantize_fn(nn.Module):
    def __init__(self, w_bit):
        super(weight_quantize_fn, self).__init__()
        self.w_bit = w_bit-1
        self.weight_q = weight_quantization(b=self.w_bit)
        self.wgt_alpha = 0.0

    def forward(self, weight):
        weight_q = self.weight_q(weight, self.wgt_alpha)

        return weight_q

Find min-max values of tensors.

In [8]:
for k,v in model["generator"].items():
    print(torch.max(v),torch.min(v))

tensor(1.1634, device='cuda:0') tensor(-0.9768, device='cuda:0')
tensor(0.1576, device='cuda:0') tensor(-0.1323, device='cuda:0')
tensor(1.1748, device='cuda:0') tensor(0.6238, device='cuda:0')
tensor(0.1959, device='cuda:0') tensor(-0.5657, device='cuda:0')
tensor(4.7445, device='cuda:0') tensor(-4.7815, device='cuda:0')
tensor(7.3949, device='cuda:0') tensor(0.3573, device='cuda:0')
tensor(0, device='cuda:0') tensor(0, device='cuda:0')
tensor(0.6126, device='cuda:0') tensor(-0.6328, device='cuda:0')
tensor(0.0594, device='cuda:0') tensor(-0.0617, device='cuda:0')
tensor(1.1618, device='cuda:0') tensor(0.4672, device='cuda:0')
tensor(-0.0327, device='cuda:0') tensor(-1.8667, device='cuda:0')
tensor(7.5120, device='cuda:0') tensor(-11.9504, device='cuda:0')
tensor(62.0158, device='cuda:0') tensor(5.6193, device='cuda:0')
tensor(0, device='cuda:0') tensor(0, device='cuda:0')
tensor(0.5240, device='cuda:0') tensor(-0.5447, device='cuda:0')
tensor(0.0246, device='cuda:0') tensor(-0.0257, 

Quantizing Model (Function call to quantization function)
Note-
1.set weight alpa to max values of tensors.[Can play with this]
2. w_bit takes the precision values can change this to see changes.

In [15]:
quantized_dict={}
for k,v in model["generator"].items():
    if "weight" or "bias" in k:
        weight_quantize=weight_quantize_fn(w_bit=8)
        weight_quantize.wgt_alpha = torch.tensor(torch.max(v))
        ouput_quantized=weight_quantize(v)
        w_delta      = torch.max(v)/(2**(8-1)-1)
        w_int        = ouput_quantized/w_delta
        quantized_dict[k]=w_int
print(quantized_dict)

<ipython-input-15-9cbda3493ab0>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weight_quantize.wgt_alpha = torch.tensor(torch.max(v))


{'dense_motion_network.hourglass.encoder.down_blocks.0.conv.weight': tensor([[[[[  2.0000,  -1.0000,   0.0000],
           [  0.0000,   0.0000,  -2.0000],
           [  2.0000,   0.0000,  -1.0000]],

          [[ -2.0000,   0.0000,   2.0000],
           [ -1.0000,   0.0000,  -2.0000],
           [ -2.0000,   2.0000,  -1.0000]],

          [[  1.0000,   0.0000,   1.0000],
           [  0.0000,   0.0000,   1.0000],
           [  2.0000,   0.0000,   0.0000]]],


         [[[ 35.0000,  26.0000,  28.0000],
           [ 44.0000,  38.0000,  35.0000],
           [ 20.0000,  12.0000,  12.0000]],

          [[ 32.0000,  24.0000,  24.0000],
           [ 38.0000,  30.0000,  32.0000],
           [ 14.0000,   9.0000,   8.0000]],

          [[ 27.0000,  17.0000,  20.0000],
           [ 38.0000,  27.0000,  26.0000],
           [ 13.0000,   8.0000,   4.0000]]],


         [[[ 10.0000,  14.0000,   6.0000],
           [ 10.0000,  19.0000,  11.0000],
           [  4.0000,  11.0000,   8.0000]],

          

Update model dictionary with updated quantized weights.

In [11]:
for k,v in model["generator"].items():
    model['generator'][k] = quantized_dict[k]

Save the updated model state dictionary to checkpoint

In [12]:
PATH = "model.pth.tar"
torch.save(model, PATH)